# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [388]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [389]:
aviation_df = pd.read_csv(
    r'c:\Users\steve\Desktop\Flatiron\Week2\dsc-course0-m8-lab-main\dsc-course0-m8-lab-main\data\AviationData.csv',
    encoding='latin1',
    index_col=0
)

C:\Users\steve\AppData\Local\Temp\ipykernel_12092\214533235.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  aviation_df = pd.read_csv(


In [390]:
# inspect NaNs
print(aviation_df.isna().sum().sort_values(ascending=False))

Schedule                  76307
Air.carrier               72241
FAR.Description           56866
Aircraft.Category         56602
Longitude                 54516
Latitude                  54507
Airport.Code              38757
Airport.Name              36185
Broad.phase.of.flight     27165
Publication.Date          13771
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Fatal.Injuries      11401
Engine.Type                7096
Report.Status              6384
Purpose.of.flight          6192
Number.of.Engines          6084
Total.Uninjured            5912
Weather.Condition          4492
Aircraft.damage            3194
Registration.Number        1382
Injury.Severity            1000
Country                     226
Amateur.Built               102
Model                        92
Make                         63
Location                     52
Accident.Number               0
Event.Date                    0
Investigation.Type            0
dtype: int64


In [391]:
aviation_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88889 entries, 20001218X45444 to 20221230106513
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Investigation.Type      88889 non-null  object 
 1   Accident.Number         88889 non-null  object 
 2   Event.Date              88889 non-null  object 
 3   Location                88837 non-null  object 
 4   Country                 88663 non-null  object 
 5   Latitude                34382 non-null  object 
 6   Longitude               34373 non-null  object 
 7   Airport.Code            50132 non-null  object 
 8   Airport.Name            52704 non-null  object 
 9   Injury.Severity         87889 non-null  object 
 10  Aircraft.damage         85695 non-null  object 
 11  Aircraft.Category       32287 non-null  object 
 12  Registration.Number     87507 non-null  object 
 13  Make                    88826 non-null  object 
 14  Model                

## Data Cleaning

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

### Filtering aircrafts and events

In [392]:
aviation_df.columns

Index(['Investigation.Type', 'Accident.Number', 'Event.Date', 'Location',
       'Country', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name',
       'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category',
       'Registration.Number', 'Make', 'Model', 'Amateur.Built',
       'Number.of.Engines', 'Engine.Type', 'FAR.Description', 'Schedule',
       'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [393]:
columns_to_drop = [
    'Accident.Number', 'Location', 'Country', 'Latitude', 'Longitude',
    'Airport.Code', 'Airport.Name', 'Registration.Number', 'Air.carrier',
    'Report.Status', 'Publication.Date'
]   #Removing irrelavant data in respect to the prompt

aviation_cleaned_df = aviation_df.copy().drop(columns=columns_to_drop)
aviation_cleaned_df = aviation_cleaned_df.reset_index()

In [394]:
aviation_cleaned_df['Event.Date'] = pd.to_datetime(aviation_cleaned_df['Event.Date'], errors='coerce')
aviation_cleaned_df = aviation_cleaned_df[aviation_cleaned_df['Event.Date'].dt.year >= 1983] #include only Event.Date above 1983

In [395]:
aviation_cleaned_df.columns

Index(['Event.Id', 'Investigation.Type', 'Event.Date', 'Injury.Severity',
       'Aircraft.damage', 'Aircraft.Category', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight'],
      dtype='object')

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [396]:
injury_cols = [
    'Total.Fatal.Injuries', 
    'Total.Serious.Injuries', 
    'Total.Minor.Injuries', 
    'Total.Uninjured'
]

for col in injury_cols:
    aviation_cleaned_df[col] = aviation_cleaned_df[col].fillna(0).astype(int)

for col in injury_cols:
    # Replace empty strings and whitespace with NaN, then fill NaN with 0 in the 4 columns above
    aviation_cleaned_df[col] = (
        aviation_cleaned_df[col]
        .replace(r'^\s*$', pd.NA, regex=True)
        .astype('float')
        .fillna(0)
    )

In [397]:
# Create 'Total.Passengers' and cast to integer
aviation_cleaned_df['Total.Passengers'] = (
    aviation_cleaned_df['Total.Fatal.Injuries'] +
    aviation_cleaned_df['Total.Serious.Injuries'] +
    aviation_cleaned_df['Total.Minor.Injuries'] +
    aviation_cleaned_df['Total.Uninjured']
).astype(int)

# Create 'Total.Fatal/Serious_injuries' and cast to integer
aviation_cleaned_df['Total.Fatal/Serious_injuries'] = (
    aviation_cleaned_df['Total.Fatal.Injuries'] +
    aviation_cleaned_df['Total.Serious.Injuries']
).astype(int)

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [398]:
# Clean 'Aircraft.damage' column, converting blanks and "Nan" strings to "Unknown"
aviation_cleaned_df['Aircraft.damage'] = (
    aviation_cleaned_df['Aircraft.damage']
    .astype(str)
    .str.strip()
    .str.title()
    .replace([r'^\s*$', 'Nan', 'NaN'], 'Unknown', regex=True)
)

# View cleaned value counts
print(aviation_cleaned_df['Aircraft.damage'].value_counts(dropna=False))

Aircraft.damage
Substantial    61775
Destroyed      17575
Unknown         3257
Minor           2682
Name: count, dtype: int64


In [399]:
aviation_cleaned_df['Aircraft.destroyed'] = aviation_cleaned_df['Aircraft.damage'] == 'Destroyed'
#Creates Aircraft.destroyed column based on if Aircraft.Damage = Destroyed
print(aviation_cleaned_df['Aircraft.destroyed'].value_counts(dropna=False))

Aircraft.destroyed
False    67714
True     17575
Name: count, dtype: int64


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [400]:
aviation_cleaned_df['Make'] = aviation_cleaned_df['Make'].astype(str).str.strip().str.title()
#Strip whitespace and standardize casing

In [401]:
aviation_cleaned_df['Make'] = (
    aviation_cleaned_df['Make']
    .astype(str)                      # Ensure all entries are strings
    .str.replace(',', '', regex=False)    # Remove commas
    .str.replace('.', '', regex=False)    # Remove periods
    .str.replace('-', ' ', regex=False)    # Reomve hyphen
    .str.replace('/', ' ', regex=False)   # Replace slashes with spaces
    .str.strip()                      # Trim leading/trailing whitespace
)


In [402]:
#  Drop rows with missing Make or Total.Passengers
aviation_cleaned_df = aviation_cleaned_df.dropna(subset=['Make', 'Total.Passengers'])

# Ensure Total.Passengers is numeric
aviation_cleaned_df['Total.Passengers'] = pd.to_numeric(
    aviation_cleaned_df['Total.Passengers'], errors='coerce'
)

aviation_cleaned_df = aviation_cleaned_df[
    ~aviation_cleaned_df['Make'].isin(['', 'Nan', 'NaN'])
]

make_replacements = {
    'Bombardier Inc': 'Bombardier',
    'De Havilland': 'Dehavilland',
    'Mcdonnell Douglas Aircraft Co': 'Mcdonnell Douglas',
    'Saab Scania Ab (Saab)': 'Saab'
}

aviation_cleaned_df['Make'] = aviation_cleaned_df['Make'].replace(make_replacements)

# Aggregate total passengers by Make
passenger_by_make = (
    aviation_cleaned_df.groupby('Make')['Total.Passengers']
    .sum()
    .sort_values(ascending=False)
    .head(50)
)

# Filter to only those top 50 makes
aviation_cleaned_df = aviation_cleaned_df[
    aviation_cleaned_df['Make'].isin(passenger_by_make.index)
]

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [403]:
aviation_cleaned_df['Model'] = (
    aviation_cleaned_df['Model']
    .astype(str)
    .str.strip()
    .str.upper()
)

aviation_cleaned_df['Model'] = (
    aviation_cleaned_df['Model']
    .astype(str)                      # Ensure all entries are strings
    .str.replace(',', '', regex=False)    # Remove commas
    .str.replace('.', '', regex=False)    # Remove periods
    .str.replace('-', ' ', regex=False)    # Reomve hyphen
    .str.replace('/', ' ', regex=False)   # Replace slashes with spaces
    .str.strip()                      # Trim leading/trailing whitespace
)

# Remove NaN
aviation_cleaned_df = aviation_cleaned_df.dropna(subset=['Model'])

In [404]:
import numpy as np
import re

# Standardize and compare Make and Model in a case-insensitive, whitespace-trimmed way
same_make_model = (
    aviation_cleaned_df['Make'].str.strip().str.upper() == 
    aviation_cleaned_df['Model'].str.strip().str.upper()
)

# Create the Aircraft_make_model column based on the comparison
aviation_cleaned_df['Aircraft.Make_Model'] = np.where(
    same_make_model,
    aviation_cleaned_df['Make'],  # If same, just use Make
    aviation_cleaned_df['Make'] + ' ' + aviation_cleaned_df['Model']  # Else, concatenate
)

# Pattern explanation:
#   - (\d)\s+(\d): captures a digit followed by spaces followed by another digit
#   - (?=\s*\S*$): ensures this pattern occurs only in the **last two words** of the string
#   - Ensures that Airbus A340  300 and Airbus A340300 are the same
aviation_cleaned_df['Aircraft.Make_Model'] = aviation_cleaned_df['Aircraft.Make_Model'].str.replace(
    r'(\d)\s+(\d)(?=\s*\S*$)', r'\1\2', regex=True
)



### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [405]:
# Define a standard cleaning function for categorical columns
def clean_column(col, to_case='title'):
    col = col.astype(str).str.strip()
    if to_case == 'title':
        return col.str.title()
    elif to_case == 'upper':
        return col.str.upper()
    return col  # No case conversion

# Engine.Type: clean and replace string 'Nan' with 'Unknown'
aviation_cleaned_df['Engine.Type'] = clean_column(aviation_cleaned_df['Engine.Type'], to_case='title')
aviation_cleaned_df['Engine.Type'] = aviation_cleaned_df['Engine.Type'].replace(['Nan', 'NaN', 'nan'], 'Unknown')

# Weather.Condition: clean (already mostly upper case categories like VMC, IMC)
aviation_cleaned_df['Weather.Condition'] = clean_column(aviation_cleaned_df['Weather.Condition'], to_case='upper')

# Purpose.of.flight: clean and replace string 'Nan' with 'Unknown'
aviation_cleaned_df['Purpose.of.flight'] = clean_column(aviation_cleaned_df['Purpose.of.flight'], to_case='title')
aviation_cleaned_df['Purpose.of.flight'] = aviation_cleaned_df['Purpose.of.flight'].replace(['Nan', 'NaN', 'nan'], 'Unknown')

# Broad.phase.of.flight: clean and replace string 'Nan' with 'Unknown'
aviation_cleaned_df['Broad.phase.of.flight'] = clean_column(aviation_cleaned_df['Broad.phase.of.flight'], to_case='title')
aviation_cleaned_df['Broad.phase.of.flight'] = aviation_cleaned_df['Broad.phase.of.flight'].replace(['Nan', 'NaN', 'nan'], 'Unknown')

# Number.of.Engines: convert to numeric (non-convertible values become NaN)
aviation_cleaned_df['Number.of.Engines'] = pd.to_numeric(
    aviation_cleaned_df['Number.of.Engines'], errors='coerce'
)

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [406]:
#Inspect # of NaNs
aviation_cleaned_df.isna().sum().sort_values(ascending=False)

Schedule                        56134
FAR.Description                 47095
Aircraft.Category               46910
Number.of.Engines                4470
Injury.Severity                   888
Amateur.Built                      86
Total.Serious.Injuries              0
Aircraft.destroyed                  0
Total.Fatal/Serious_injuries        0
Total.Passengers                    0
Broad.phase.of.flight               0
Weather.Condition                   0
Total.Uninjured                     0
Total.Minor.Injuries                0
Event.Id                            0
Total.Fatal.Injuries                0
Purpose.of.flight                   0
Investigation.Type                  0
Engine.Type                         0
Model                               0
Make                                0
Aircraft.damage                     0
Event.Date                          0
Aircraft.Make_Model                 0
dtype: int64

In [407]:
#drop columns with many NaNs
aviation_cleaned_df.drop(
    columns=['Schedule', 'FAR.Description', 'Aircraft.Category'], 
    inplace=True
)

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [408]:
aviation_cleaned_df.to_csv('AviationData_cleaned.csv', index=False)